<a href="https://colab.research.google.com/github/osmarbraz/sri/blob/main/1_1_Segmentacao_Limpeza_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentação e limpeza

Realiza a segmentação e limpeza de um conjunto de dados em sua forma bruta.

**Entrada**: "`documentos.csv`".
 - Cada linha do arquivo é formado por `["id","documento"]`.
  - `"id"` é o idenficador do documento.
  - `"documento"` o documento texto no seu formato bruto.

**Saída**: "`dataset.zip`".
 - Dentro do arquivo compactado `dataset.zip` está o arquivo `dataset.csv`. Cada linha de `dataset.csv` é formado por `["id","sentencas","documento"]`.
  - `"id"` é o idenficador do documento na base de dados.
  - `"sentencas"` é uma lista com as sentenças do documento.
  - `"documento"` o documento limpo, mas não segmentado.

**Processamento**:
1. Copia o arquivo "`documentos.csv`" para a máquina local do
Google Colab.
2. Realiza as seguintes tarefas de limpeza:
    - Normalização de codificação e decodificação de html entities (ftfy);
    - Substitui \n por espaço em branco;
    - Elimina pontuações repetidas ("???","!!!",",,,");
    - Elimina de espaços em branco repetidos;
    - Elimina tags.
    - **Local para outras operações de limpeza.**
3. Realiza a segmentação.
4. Gera o arquivo "`dataset.csv`" com os dados pré-processados.
5. Compacta o arquivo "`dataset.csv`" para "`dataset.zip`".
6. Copia o arquivo "`dataset.zip`" para o google drive.

**Atenção - Para executar este notebook e os próximos siga as instruções abaixo.**

1. Crie a pasta "`Colab Notebooks`" na raiz do seu google drive para receber pastas de projetos.
2. Dentro da pasta "`Colab Notebooks`" crie a pasta "`SRI`" para armazenar e executar os notebooks das atividades práticas da disciplina.
3. Dentro da pasta "`SRI`" crie a pasta "data" e coloque o arquivo de dados "`documentos.csv`".

**Testes:**

Os arquivos "`documentos.csv`" e "`dataset.csv`" precisam ter ao final da execução a mesma quantidade de linhas. Onde cada linha de  "`dataset.csv`" esta relacionada com a mesma linha de "`documentos.csv`".


# 1 Preparação do ambiente

Preparação do ambiente para execução do script.

## 1.1 Tempo inicial de processamento

In [ ]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório do notebook no diretório corrente.   


In [ ]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================
def verificaDiretorioNotebook():
    """
      Verifica se existe o diretório do notebook no diretório corrente.
    """

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_NOTEBOOK):
        # Cria o diretório
        os.makedirs(DIRETORIO_NOTEBOOK)
        logging.info("Diretório do notebook criado: {}".format(DIRETORIO_NOTEBOOK))

    return DIRETORIO_NOTEBOOK

Remove tags de um documento

In [ ]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """

    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempoArredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempoArredondado))

Classe(ModelosParametros) de definição dos parâmetros dos modelos

In [ ]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModelosParametros:
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )

    sentenciar_documento: bool = field(
        default=True,
        metadata={"help": "Dividir o documento em sentenças(frases)."},
    )

Biblioteca de limpeza de tela


In [ ]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [ ]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4 Identificando o ambiente Colab

In [ ]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [ ]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [ ]:
# Import das bibliotecas.
import torch

def getDeviceGPU():
    """
      Retorna um dispositivo de GPU se disponível ou CPU.

      Retorno:
        `device` - Um device de GPU ou CPU.
    """

    # Se existe GPU disponível.
    if torch.cuda.is_available():

        # Diz ao PyTorch para usar GPU.
        device = torch.device("cuda")

        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")

    return device

In [ ]:
# Recupera o device com GPU ou CPU
device = getDeviceGPU()

Memória

Memória disponível no ambiente

In [ ]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  # import necessário
  from google.colab import drive

  # Monta o drive na pasta especificada
  drive.mount("/content/drive")

## 1.7 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

Uso:
https://spacy.io/usage

In [ ]:
# Instala dependências do spacy
!pip install -U pip==25.1.1 setuptools==80.9.0 wheel==0.45.1

In [ ]:
# Instala uma versão específica
!pip install -U spacy==3.8.7

## 1.8 Instalação do ftfy

Realiza a normalização do texto devido a problemas de codificação de caracteres e decodificação de html entities.

https://pypi.org/project/ftfy/

In [ ]:
!pip install ftfy==6.3.1

# 2 Parametrização

## Gerais

In [ ]:
# Definição dos parâmetros gerais

## Específicos

Parâmetros do modelo

In [ ]:
# Definição dos parâmetros do Modelo.
model_args = ModelosParametros(

    #modelo_spacy = "en_core_web_lg",
    #modelo_spacy = "en_core_web_md",
    #modelo_spacy = "en_core_web_sm",

    modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    #modelo_spacy = "pt_core_news_sm",

    sentenciar_documento = True,
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do notebook
DIRETORIO_NOTEBOOK = "SRI"

## Define o caminho para os arquivos de dados

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  # Diretório local para os arquivos de dados
  DIRETORIO_LOCAL = "/content/" + DIRETORIO_NOTEBOOK + "/"

  # Diretório no google drive com os arquivos de dados
  DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/" + DIRETORIO_NOTEBOOK + "/data/"
else:

  # Diretório local para os arquivos de dados
  DIRETORIO_LOCAL = "./data/"

  # Diretório no google drive com os arquivos de dados
  DIRETORIO_DRIVE = "./data/"

# 3 spaCy

## 3.1 Download arquivo modelo

Uso:
https://spacy.io/usage

Modelos:
https://spacy.io/models

In [ ]:
!python -m spacy download $model_args.modelo_spacy

## 3.2 Carrega o modelo

In [ ]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

nlp = spacy.load(model_args.modelo_spacy)

# 4 Pré-processamento do arquivo "`documentos.csv`"


## 4.1 Carrega os dados bruto

### 4.1.1 Especifica s nome do arquivo de dados bruto



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_DADOS_BRUTO = "documentos.csv"

### 4.1.2 Cria o diretório local para receber os arquivos de dados

In [ ]:
# Importando as bibliotecas.
import os

# Se estiver executando no Google Colaboratory
if IN_COLAB:

  # Cria o diretório para receber os arquivos Originais e Permutados
  # Diretório a ser criado
  dirbase = DIRETORIO_LOCAL[:-1]

  if not os.path.exists(dirbase):
      # Cria o diretório
      os.makedirs(dirbase)
      logging.info("Diretório criado: {}".format(dirbase))
  else:
      logging.info("Diretório já existe: {}".format(dirbase))

### 4.1.3 Copia o arquivos de dados bruto do google drive para o diretório local

In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_DADOS_BRUTO" "$DIRETORIO_LOCAL"

  logging.info("Terminei a cópia!")

### 4.1.4 Carrega os dados no formato bruto

Atributos do arquivo **documentos**:
0. "id"
1. "documento"

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
df_documentos_bruto = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DADOS_BRUTO, sep=";", encoding="UTF-8")

logging.info("TERMINADO FONTES: {}.".format(len(df_documentos_bruto)))

In [ ]:
df_documentos_bruto.sample(5)

## 4.2 Segmentação e limpeza dos dados bruto

### 4.2.1 Limpeza do documento

- Normalização de codificação e decodificação de html entities (ftfy);
- Substitui \n por espaço em branco;
- Eliminar pontuações repetidas ("???","!!!",",,,");
- Eliminar de espaços em branco repetidos;
- Eliminar tags;
- **>>> Outras operações de limpeza podem ser adicionadas aqui.<<<**.


Limpeza de tags html: https://pypi.org/project/beautifulsoup4/ ou
https://medium.com/pyladiesbh/beautiful-soup-parseamento-de-html-337197a7d4b9


In [ ]:
# Import das bibliotecas,
import ftfy # Biblioteca para normalização do texto
import re # Biblioteca para expressões regulares
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

# Lista para os documentos tratados
documentos_tratados = []

# Acumula o total de sentenças
conta_barra_n = 0
conta_virgulas = 0
conta_interrogacoes = 0
conta_exclamacoes = 0
conta_tags = 0
conta_espacos = 0

logging.info("Processando {} documentos.".format(len(df_documentos_bruto)))

# Barra de progresso dos dados
dados_bruto_bar = tqdm_notebook(df_documentos_bruto.iterrows(), desc=f"Dados", unit=f"registro", total=len(df_documentos_bruto))

# Percorre os registros dos dados bruto
for (i, linha) in dados_bruto_bar:

  # Recupera o id documento
  id_documento = linha.values[0]

  # Recupera o documento
  documento = str(linha.values[1])

  # Transforma em string e remove os espaços do início e do fim
  documento = str(documento).strip()

  # Normalização com o ftfy, e conversão de html entities em caracteres acentuados
  documento = ftfy.fix_text(documento, fix_entities=True)

  # Conta se existe \n no documento
  conta_caracter_barra_n = documento.count("\n")
  if conta_caracter_barra_n > 0:
    # Transforma \n em espaços em branco
    documento = documento.replace("\n"," ")
    conta_barra_n = conta_barra_n + 1

  # Conta se existe duas interrogações no documento
  conta_caracter_interrogacoes = documento.count("??")
  if conta_caracter_interrogacoes > 1:
    # Transforma 2 ou mais interrogações consecutivas em 1
    documento = re.sub("\?+", "?", documento)
    conta_interrogacoes = conta_interrogacoes + 1

  # Conta se existe duas exclamações no documento
  conta_caracter_exclamacoes = documento.count("!!")
  if conta_caracter_exclamacoes > 1:
    # Transforma 2 ou mais exclamações consecutivas em 1
    documento = re.sub("\!+", "!", documento)
    conta_exclamacoes = conta_exclamacoes + 1

  # Conta se existe duas vírgulas no documento
  conta_caracter_virgulas = documento.count(",,")
  if conta_caracter_virgulas > 1:
    # Transforma 2 ou mais vírgulas consecutivas em 1
    documento = re.sub("\,+", ",", documento)
    conta_virgulas = conta_virgulas + 1

  # Conta se existe tag no documento
  conta_tag = documento.count("<")
  if conta_tag > 0:
    # Remove as tags do documento
    documento = re.sub("<.*?>", "", documento)
    conta_tags = conta_tags + 1

  # Conta se existe dois caracteres branco
  conta_caracter_espacos = documento.count("  ")
  if conta_caracter_espacos > 0:
    # Transforma 2 ou mais espaços consecutivos em 1
    documento = re.sub("\ +", " ", documento)
    conta_espacos = conta_espacos + 1

  # >>> Outras operações de limpeza vão aqui! <<<

  # Guarda o id e o texto do documento tratado
  documentos_tratados.append([id_documento, documento])

In [ ]:
print("Total de documentos com \\n                        :", conta_barra_n)
print("Total de documentos com 2 ou mais interrogações   :", conta_interrogacoes)
print("Total de documentos com 2 ou mais exclamações     :", conta_exclamacoes)
print("Total de documentos com 2 ou mais vírgulas        :", conta_virgulas)
print("Total de documentos com tags                      :", conta_tags)
print("Total de documentos com 2 ou mais espaços         :", conta_espacos)
print("Total de documentos tratadas                      :", len(documentos_tratados))

Converte a lista em um pandas dataframe.

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Cria o dataframe da lista
df_documentos_tratados = pd.DataFrame(documentos_tratados, columns = ["id", "documento"])

#Mostra o número de documentos carregados
print("%d linhas carregadas do arquivo" % (len(df_documentos_tratados)))

# Mostra 10 linhas aleatórias dos dados
df_documentos_tratados.sample(10)

### 4.2.2 Segmentação do documento

Utiliza o spaCy para realizar a segmentação em sentenças do documento texto.

In [ ]:
# Import das bibliotecas.
import os # Biblioteca para acessar o sistema de arquivos
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

print("Processando",len(df_documentos_tratados),"documentos tratados")

# Sentenças por documento
lista_documentos_sentenciados = []
total_sentencas_documento = 0

logging.info("Processando {} documentos.".format(len(df_documentos_tratados)))

# Barra de progresso dos dados
dados_bar = tqdm_notebook(df_documentos_tratados.iterrows(), desc=f"Dados", unit=f"registro", total=len(df_documentos_tratados))

# Percorre os registros dos documento
for i, linha_documento in dados_bar:

  # Recupera o documento
  documento = str(linha_documento.values[1])

  # Aplica sentenciação do spacy no documento
  doc = nlp(documento)

  # Sequência das sentenças no documento
  sequencia = 1
  sentencas = []

  # Percorre as sentenças do documento
  for sentenca in doc.sents:

      # Conta as sentenças do documento
      total_sentencas_documento = total_sentencas_documento + 1

      # Transforma em string e remove os espaços do início e do fim
      sentenca1 = str(sentenca).strip()

      # Adiciona a sentença tratada na lista
      sentencas.append(str(sentenca1))

      # Incrementa a sequência da sentença no documento
      sequencia = sequencia + 1

  if model_args.sentenciar_documento == True:
    # Adiciona o documento na lista com a sentenciação
    lista_documentos_sentenciados.append([linha_documento[0], sentencas, str(linha_documento[1])])
  else:
    # Adiciona o documento na lista sem a sentenciação
    # A lista de sentenças é formado por um único texto, o próprio documento.
    lista_documentos_sentenciados.append([linha_documento[0], [str(linha_documento[1])], str(linha_documento[1])])


In [ ]:
print("Total de documentos processados                                                :", len(lista_documentos_sentenciados))
print("Total sentenças nos documentos usando spaCy                                    :", total_sentencas_documento)

## 4.3 Salva os dados processados

Gera o arquivo com os dados segmentados e limpos e depois compacta o arquivo para enviar para o Google Drive.

### 4.3.1 Especifica os nomes dos arquivos do dataset



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_DATASET = "dataset.csv"
NOME_ARQUIVO_DATASET_COMPACTADO = "dataset.zip"

### 4.3.2 Cria o arquivo do dataset

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Cria o dataframe da lista
df_lista_documentos_sentenciados = pd.DataFrame(lista_documentos_sentenciados, columns = ["id","sentencas","documento"])

df_lista_documentos_sentenciados.to_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DATASET, sep=";", index=False)

In [ ]:
print(len(df_lista_documentos_sentenciados))

In [ ]:
df_lista_documentos_sentenciados.sample(5)

### 4.3.3 Compacta e copia o dataset para uma pasta do GoogleDrive

Compacta os arquivos.

Usa o zip para compactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens


In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !zip -o -q -j "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET"

Copia o arquivo compactado para o GoogleDrive



In [ ]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
    # Copia o arquivo do dataset
    !cp "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_COMPACTADO" "$DIRETORIO_DRIVE"

    logging.info("Terminei a cópia")

### 4.3.4 Carrega os dados

Realiza um teste carregando o arquivo do dataset criado.

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
df_dataset = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DATASET, sep=";", encoding="UTF-8")

Corrigir o tipo de dados da lista dos documentos

Na lista
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
df_dataset["sentencas"] = df_dataset["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO DOCUMENTOS: {}.".format(len(df_dataset)))

In [ ]:
df_dataset.sample(5)

# 5 Finalização

## 5.1 Tempo final de processamento



In [ ]:
# Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempo_total_processamento))